In [1]:
#Defining the Node Class

import random
import time

class Node:
    def __init__(self, node_id, ip_address):
        self.node_id = node_id
        self.ip_address = ip_address
        self.routing_table = {}  # destination -> (next_hop, seq_num, hop_count)
        self.neighbors = []  # List of neighbor nodes
        self.seq_num = random.randint(0, 1000)  # Node sequence number

    def add_neighbor(self, neighbor_node):
        """Add a neighbor to this node's neighbor list."""
        if neighbor_node not in self.neighbors:
            self.neighbors.append(neighbor_node)

    def send_rreq(self, destination):
        """Send a route request (RREQ) to discover the route to the destination."""
        print(f"Node {self.node_id} sending RREQ to {destination.node_id}.")
        rreq = {
            'sender': self,
            'destination': destination,
            'seq_num': self.seq_num,
            'type': 'RREQ'
        }
        self.broadcast_rreq(rreq)

    def broadcast_rreq(self, rreq):
        """Broadcast the RREQ to all neighbors."""
        for neighbor in self.neighbors:
            if neighbor != rreq['sender']:
                neighbor.receive_rreq(rreq)

    def receive_rreq(self, rreq):
        """Process an incoming RREQ."""
        if self.node_id == rreq['destination'].node_id:
            # If the destination node receives the RREQ, send RREP
            print(f"Node {self.node_id} received RREQ from Node {rreq['sender'].node_id}. Sending RREP.")
            rrep = {
                'sender': self,
                'destination': rreq['sender'],
                'seq_num': self.seq_num,
                'type': 'RREP'
            }
            self.send_rrep(rrep)
        else:
            # If not the destination, forward the RREQ to neighbors
            if rreq['destination'] not in self.routing_table:
                self.routing_table[rreq['destination'].node_id] = (rreq['sender'], self.seq_num, 1)
            self.broadcast_rreq(rreq)

    def send_rrep(self, rrep):
        """Send a route reply (RREP) to the sender."""
        print(f"Node {self.node_id} sending RREP to Node {rrep['sender'].node_id}.")
        rrep['sender'].receive_rrep(rrep)

    def receive_rrep(self, rrep):
        """Process an incoming RREP."""
        if self.node_id != rrep['sender'].node_id:
            print(f"Node {self.node_id} received RREP from Node {rrep['sender'].node_id}.")
            self.routing_table[rrep['sender'].node_id] = (rrep['sender'], self.seq_num, 1)

    def send_message(self, destination, message):
        """Send a message to the destination node using the routing table."""
        if destination.node_id in self.routing_table:
            print(f"Node {self.node_id} sending message to Node {destination.node_id}: {message}")
            destination.receive_message(self, message)
        else:
            print(f"Node {self.node_id} does not have a route to Node {destination.node_id}. Initiating RREQ.")
            self.send_rreq(destination)

    def receive_message(self, sender, message):
        """Receive a message."""
        print(f"Node {self.node_id} received message: '{message}' from Node {sender.node_id}")

    def __str__(self):
        return f"Node {self.node_id} ({self.ip_address})"


In [2]:
# Defining Adhoc Network Class

class AdHocNetwork:
    def __init__(self):
        self.nodes = []  # List of nodes in the network

    def add_node(self, node):
        """Add a node to the network."""
        self.nodes.append(node)

    def connect_nodes(self, node1, node2):
        """Establish a neighbor relationship between two nodes."""
        node1.add_neighbor(node2)
        node2.add_neighbor(node1)

    def send_message_from_node(self, sender_node, destination_node, message):
        """Send a message from one node to another."""
        sender_node.send_message(destination_node, message)

    def display_network(self):
        """Display the nodes and their neighbors."""
        for node in self.nodes:
            neighbors = [str(neighbor.node_id) for neighbor in node.neighbors]
            print(f"Node {node.node_id} ({node.ip_address}) has neighbors: {', '.join(neighbors)}")


In [ ]:
# Example Simulation

def run_simulation():
    # Create nodes
    node1 = Node(1, "192.168.0.1")
    node2 = Node(2, "192.168.0.2")
    node3 = Node(3, "192.168.0.3")
    node4 = Node(4, "192.168.0.4")
    node5 = Node(5, "192.168.0.5")

    # Create ad-hoc network
    network = AdHocNetwork()
    network.add_node(node1)
    network.add_node(node2)
    network.add_node(node3)
    network.add_node(node4)
    network.add_node(node5)

    # Connect nodes (establish neighbor relationships)
    network.connect_nodes(node1, node2)
    network.connect_nodes(node2, node3)
    network.connect_nodes(node3, node4)
    network.connect_nodes(node4, node5)

    # Display the network setup
    network.display_network()

    # Simulate communication: Node 1 sends a message to Node 5
    print("\n--- Sending message from Node 1 to Node 5 ---")
    network.send_message_from_node(node1, node5, "Hello Node 5, this is Node 1.")

    # Simulate a second communication: Node 2 sends a message to Node 4
    print("\n--- Sending message from Node 2 to Node 4 ---")
    network.send_message_from_node(node2, node4, "Hello Node 4, this is Node 2.")

if __name__ == "__main__":
    run_simulation()

Node 1 (192.168.0.1) has neighbors: 2
Node 2 (192.168.0.2) has neighbors: 1, 3
Node 3 (192.168.0.3) has neighbors: 2, 4
Node 4 (192.168.0.4) has neighbors: 3, 5
Node 5 (192.168.0.5) has neighbors: 4

--- Sending message from Node 1 to Node 5 ---
Node 1 does not have a route to Node 5. Initiating RREQ.
Node 1 sending RREQ to 5.


RecursionError: maximum recursion depth exceeded